In [38]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage
from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [39]:
nfl_data = nfl.import_schedules([2023])
original_df = nfl_data.groupby('week')
original_df.head()

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6421,2023_01_DET_KC,2023,REG,1,2023-09-07,Thursday,20:20,DET,21.0,KC,...,NaN,00-0033106,00-0033873,Jared Goff,Patrick Mahomes,Dan Campbell,Andy Reid,John Hussey,KAN00,GEHA Field at Arrowhead Stadium
6422,2023_01_CAR_ATL,2023,REG,1,2023-09-10,Sunday,13:00,CAR,10.0,ATL,...,NaN,00-0039150,00-0038122,Bryce Young,Desmond Ridder,Frank Reich,Arthur Smith,Brad Rogers,ATL97,Mercedes-Benz Stadium
6423,2023_01_HOU_BAL,2023,REG,1,2023-09-10,Sunday,13:00,HOU,9.0,BAL,...,NaN,00-0039163,00-0034796,C.J. Stroud,Lamar Jackson,DeMeco Ryans,John Harbaugh,Tra Blake,BAL00,M&T Bank Stadium
6424,2023_01_CIN_CLE,2023,REG,1,2023-09-10,Sunday,13:00,CIN,3.0,CLE,...,NaN,00-0036442,00-0033537,Joe Burrow,Deshaun Watson,Zac Taylor,Kevin Stefanski,Clete Blakeman,CLE00,FirstEnergy Stadium
6425,2023_01_JAX_IND,2023,REG,1,2023-09-10,Sunday,13:00,JAX,31.0,IND,...,NaN,00-0036971,00-0039164,Trevor Lawrence,Anthony Richardson,Doug Pederson,Shane Steichen,Clay Martin,IND00,Lucas Oil Stadium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6677,2023_18_SEA_ARI,2023,REG,18,2024-01-07,Sunday,13:00,SEA,NaN,ARI,...,NaN,NaN,NaN,NaN,NaN,Pete Carroll,Jonathan Gannon,NaN,PHO00,State Farm Stadium
6678,2023_18_PIT_BAL,2023,REG,18,2024-01-07,Sunday,13:00,PIT,NaN,BAL,...,NaN,NaN,NaN,NaN,NaN,Mike Tomlin,John Harbaugh,NaN,BAL00,M&T Bank Stadium
6679,2023_18_TB_CAR,2023,REG,18,2024-01-07,Sunday,13:00,TB,NaN,CAR,...,NaN,NaN,NaN,NaN,NaN,Todd Bowles,Frank Reich,NaN,CAR00,Bank of America Stadium
6680,2023_18_CLE_CIN,2023,REG,18,2024-01-07,Sunday,13:00,CLE,NaN,CIN,...,NaN,NaN,NaN,NaN,NaN,Kevin Stefanski,Zac Taylor,NaN,CIN00,Paycor Stadium


In [40]:
# Select relevant columns for team vs. team DataFrame
team_vs_team_df = nfl_data[['season', 'week', 'away_team', 'away_score', 'home_team', 'home_score']]

# Rename the columns for clarity
team_vs_team_df.columns = ['season', 'week', 'team_1', 'score_1', 'team_2', 'score_2']

# Drop rows with missing scores
#team_vs_team_df = team_vs_team_df.dropna(subset=['score_1', 'score_2'])

# Reset the index of the new DataFrame
team_vs_team_df.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
team_vs_team_df.head()

,season,week,team_1,score_1,team_2,score_2
0,2023,1,DET,21.0,KC,20.0
1,2023,1,CAR,10.0,ATL,24.0
2,2023,1,HOU,9.0,BAL,25.0
3,2023,1,CIN,3.0,CLE,24.0
4,2023,1,JAX,31.0,IND,21.0


In [43]:
#defensive epa stats
weekly_nfl = nfl.import_weekly_data([2023])
# Create DF to get epa rushing and receiving vs d
columns = ['opponent_team', 'receiving_epa', 'rushing_epa','passing_yards','rushing_yards','pacr']
epa_df = weekly_nfl[columns]
# Replace all 0 values with NaN
epa_df = epa_df.groupby(['opponent_team']).apply(lambda group: group.mask((group == 0) | group.isna()).mean()).reset_index()
#epa_sum_df = epa_df[epa_df['opponent_team'] == 'LV']
epa_sum_df = epa_df.groupby('opponent_team').mean().reset_index()
epa_sum_df.head()

Downcasting floats.


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


,opponent_team,receiving_epa,rushing_epa,passing_yards,rushing_yards,pacr
0,ARI,1.443135,0.147999,274.399994,29.571428,1.128530
1,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493
2,BAL,-0.012423,-1.285270,207.199997,24.052631,0.743376
3,BUF,0.225897,-0.089751,218.399994,30.454546,1.166543
4,CAR,0.554338,1.034495,170.333328,44.000000,1.259308


In [42]:
#offensive epa stats
weekly_nfl = nfl.import_weekly_data([2023])
# Create DF to get epa rushing and receiving per team
columns_o = ['recent_team', 'receiving_epa', 'rushing_epa','passing_yards','rushing_yards','pacr']
epa_o_df = weekly_nfl[columns_o]
# Replace all 0 values with NaN
epa_o_df = epa_o_df.groupby(['recent_team']).apply(lambda group: group.mask((group == 0) | group.isna()).mean()).reset_index()
epa_o_df = epa_o_df.groupby('recent_team').mean().reset_index()
epa_o_df.head(15)


Downcasting floats.


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


,recent_team,receiving_epa,rushing_epa,passing_yards,rushing_yards,pacr
0,ARI,0.366136,0.114270,196.000000,32.545456,0.840810
1,ATL,0.096836,-0.954748,182.500000,38.000000,1.188958
2,BAL,0.641419,0.109011,206.000000,31.739130,1.113971
3,BUF,1.524897,0.464762,281.399994,22.346153,1.076868
4,CAR,0.526087,-0.757637,222.199997,21.818182,0.913370
5,CHI,0.751957,0.382284,228.600006,31.142857,1.109271
6,CIN,0.205974,-1.016093,209.000000,20.722221,0.877904
7,CLE,-0.463963,-0.607535,199.750000,26.136364,0.627718
8,DAL,0.466962,-0.400317,155.571426,21.448277,2.056749
9,DEN,0.909429,-0.215516,242.000000,27.421053,1.082902


In [44]:
epa_total = pd.merge(epa_sum_df, epa_o_df, left_on='opponent_team', right_on='recent_team', how='inner')
epa_total = epa_total.drop('recent_team', axis=1)
epa_total.columns = ['team', 'defense_receiving_epa', 'defense_rushing_epa', 'defense_passing_yards','defensive_rushing_yards', 'd_pacr','receiving_epa','rushing_epa','passing_yards','rushing_yards','pacr']
epa_total['opponent'] = epa_total['team']
epa_total.head(32)

,team,defense_receiving_epa,defense_rushing_epa,defense_passing_yards,defensive_rushing_yards,d_pacr,receiving_epa,rushing_epa,passing_yards,rushing_yards,pacr,opponent
0,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.000000,32.545456,0.840810,ARI
1,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493,0.096836,-0.954748,182.500000,38.000000,1.188958,ATL
2,BAL,-0.012423,-1.285270,207.199997,24.052631,0.743376,0.641419,0.109011,206.000000,31.739130,1.113971,BAL
3,BUF,0.225897,-0.089751,218.399994,30.454546,1.166543,1.524897,0.464762,281.399994,22.346153,1.076868,BUF
4,CAR,0.554338,1.034495,170.333328,44.000000,1.259308,0.526087,-0.757637,222.199997,21.818182,0.913370,CAR
5,CHI,1.462980,-0.557780,246.000000,24.549999,1.103261,0.751957,0.382284,228.600006,31.142857,1.109271,CHI
6,CIN,0.420724,0.041143,178.000000,38.500000,0.736746,0.205974,-1.016093,209.000000,20.722221,0.877904,CIN
7,CLE,-0.563272,-1.395306,148.500000,15.944445,0.905297,-0.463963,-0.607535,199.750000,26.136364,0.627718,CLE
8,DAL,-0.260247,-0.891837,116.125000,28.045454,0.935361,0.466962,-0.400317,155.571426,21.448277,2.056749,DAL
9,DEN,1.983149,0.817700,234.833328,44.666668,1.425492,0.909429,-0.215516,242.000000,27.421053,1.082902,DEN


In [45]:
# Create a new DataFrame with 'week', 'team_1', and 'team_2' columns
teams_df = team_vs_team_df[['week', 'team_1', 'team_2']].copy()

# Rename the columns for clarity
teams_df.columns = ['week', 'team', 'opponent']

# Create a new DataFrame with 'week', 'score_1', and 'score_2' columns
scores_df = team_vs_team_df[['week', 'score_1', 'score_2']].copy()

# Rename the columns for clarity
scores_df.columns = ['week', 'team_score', 'opponent_score']

# Concatenate the two DataFrames vertically
result_df = pd.concat([teams_df, scores_df], axis=1)


In [46]:
result_df.head(30)

,week,team,opponent,week,team_score,opponent_score
0,1,DET,KC,1,21.0,20.0
1,1,CAR,ATL,1,10.0,24.0
2,1,HOU,BAL,1,9.0,25.0
3,1,CIN,CLE,1,3.0,24.0
4,1,JAX,IND,1,31.0,21.0
5,1,TB,MIN,1,20.0,17.0
6,1,TEN,NO,1,15.0,16.0
7,1,SF,PIT,1,30.0,7.0
8,1,ARI,WAS,1,16.0,20.0
9,1,GB,CHI,1,38.0,20.0


In [47]:
final_df = result_df[['week', 'team', 'opponent', 'team_score', 'opponent_score']].copy()
final_df.head(5)

,week,week,team,opponent,team_score,opponent_score
0,1,1,DET,KC,21.0,20.0
1,1,1,CAR,ATL,10.0,24.0
2,1,1,HOU,BAL,9.0,25.0
3,1,1,CIN,CLE,3.0,24.0
4,1,1,JAX,IND,31.0,21.0


In [48]:
# Duplicate the DataFrame and swap 'team' and 'opponent' columns
df2 = final_df.copy()
df2.rename(columns={'team': 'opponent', 'opponent': 'team', 'opponent_score': 'team_score', 'team_score': 'opponent_score'}, inplace=True)
order = ['team', 'opponent', 'team_score', 'opponent_score']
df2 = df2[order]
df2.head()
final_df = final_df[order]
final_df.head(30)

,team,opponent,team_score,opponent_score
0,DET,KC,21.0,20.0
1,CAR,ATL,10.0,24.0
2,HOU,BAL,9.0,25.0
3,CIN,CLE,3.0,24.0
4,JAX,IND,31.0,21.0
5,TB,MIN,20.0,17.0
6,TEN,NO,15.0,16.0
7,SF,PIT,30.0,7.0
8,ARI,WAS,16.0,20.0
9,GB,CHI,38.0,20.0


In [49]:
# Reset the index for both DataFrames
final_df = final_df.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df2.head()
# Reset the index if needed
stacked_df = pd.concat([final_df, df2], axis=0)
stacked_df.tail(20)

,team,opponent,team_score,opponent_score
252,SEA,PIT,NaN,NaN
253,DEN,LAC,NaN,NaN
254,KC,CIN,NaN,NaN
255,MIN,GB,NaN,NaN
256,ARI,SEA,NaN,NaN
257,BAL,PIT,NaN,NaN
258,CAR,TB,NaN,NaN
259,CIN,CLE,NaN,NaN
260,DET,MIN,NaN,NaN
261,GB,CHI,NaN,NaN


In [50]:
team = stacked_df.groupby('team')

In [51]:
team_and_stats = pd.merge(epa_total, stacked_df, on='team', how='inner')
team_and_stats.head(20)
team_and_stats = team_and_stats.dropna(subset=['team_score', 'opponent_score'])
#team_and_stats_two = team_and_stats(['team']).mean().reset_index()

team_and_stats.head(32)

,team,defense_receiving_epa,defense_rushing_epa,defense_passing_yards,defensive_rushing_yards,d_pacr,receiving_epa,rushing_epa,passing_yards,rushing_yards,pacr,opponent_x,opponent_y,team_score,opponent_score
0,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.000000,32.545456,0.840810,ARI,WAS,16.0,20.0
1,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.000000,32.545456,0.840810,ARI,SF,16.0,35.0
9,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.000000,32.545456,0.840810,ARI,NYG,28.0,31.0
10,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.000000,32.545456,0.840810,ARI,DAL,28.0,16.0
11,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.000000,32.545456,0.840810,ARI,CIN,20.0,34.0
17,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493,0.096836,-0.954748,182.500000,38.000000,1.188958,ATL,DET,6.0,20.0
18,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493,0.096836,-0.954748,182.500000,38.000000,1.188958,ATL,JAX,7.0,23.0
26,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493,0.096836,-0.954748,182.500000,38.000000,1.188958,ATL,CAR,24.0,10.0
27,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493,0.096836,-0.954748,182.500000,38.000000,1.188958,ATL,GB,25.0,24.0
28,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493,0.096836,-0.954748,182.500000,38.000000,1.188958,ATL,HOU,21.0,19.0


In [52]:
#######

In [53]:
team_and_stats.dtypes
team_and_stats['team'] = team_and_stats['team'].astype(object)
team_and_stats['opponent'] = team_and_stats['opponent_y'].astype(object)


In [54]:
team_and_stats.head(8)


,team,defense_receiving_epa,defense_rushing_epa,defense_passing_yards,defensive_rushing_yards,d_pacr,receiving_epa,rushing_epa,passing_yards,rushing_yards,pacr,opponent_x,opponent_y,team_score,opponent_score,opponent
0,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.0,32.545456,0.840810,ARI,WAS,16.0,20.0,WAS
1,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.0,32.545456,0.840810,ARI,SF,16.0,35.0,SF
9,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.0,32.545456,0.840810,ARI,NYG,28.0,31.0,NYG
10,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.0,32.545456,0.840810,ARI,DAL,28.0,16.0,DAL
11,ARI,1.443135,0.147999,274.399994,29.571428,1.128530,0.366136,0.114270,196.0,32.545456,0.840810,ARI,CIN,20.0,34.0,CIN
17,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493,0.096836,-0.954748,182.5,38.000000,1.188958,ATL,DET,6.0,20.0,DET
18,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493,0.096836,-0.954748,182.5,38.000000,1.188958,ATL,JAX,7.0,23.0,JAX
26,ATL,0.559732,-1.409261,199.199997,26.100000,0.820493,0.096836,-0.954748,182.5,38.000000,1.188958,ATL,CAR,24.0,10.0,CAR


In [55]:
# create opponent stats
team_and_stats.head()

team_and_stats_opp = pd.merge(epa_total, stacked_df, on='opponent', how='inner')
#ex = team_and_stats.mean().reset_index()
#team_and_stats_opp = team_and_stats_opp.dropna(axis=1, how='any')
team_and_stats_opp = team_and_stats_opp.dropna(subset=['team_score', 'opponent_score'])

team_and_stats_opp.head()

#remove_team = ['team_x', 'team_y']
#team_and_stats_opp = team_and_stats_opp.drop(columns=remove_team)

,team_x,defense_receiving_epa,defense_rushing_epa,defense_passing_yards,defensive_rushing_yards,d_pacr,receiving_epa,rushing_epa,passing_yards,rushing_yards,pacr,opponent,team_y,team_score,opponent_score
0,ARI,1.443135,0.147999,274.399994,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,ARI,NYG,31.0,28.0
1,ARI,1.443135,0.147999,274.399994,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,ARI,DAL,16.0,28.0
2,ARI,1.443135,0.147999,274.399994,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,ARI,CIN,34.0,20.0
8,ARI,1.443135,0.147999,274.399994,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,ARI,WAS,20.0,16.0
9,ARI,1.443135,0.147999,274.399994,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,ARI,SF,35.0,16.0


In [56]:
team_and_stats_test_data_two = team_and_stats.merge(team_and_stats_opp, on='opponent', how='right')
team_and_stats_test_data_two = team_and_stats_test_data_two.drop_duplicates()
#team_and_stats_test_data_two = team_and_stats_test_data.dropna(axis=1, how='any')
team_and_stats_test_data_two.head()

,team,defense_receiving_epa_x,defense_rushing_epa_x,defense_passing_yards_x,defensive_rushing_yards_x,d_pacr_x,receiving_epa_x,rushing_epa_x,passing_yards_x,rushing_yards_x,...,defensive_rushing_yards_y,d_pacr_y,receiving_epa_y,rushing_epa_y,passing_yards_y,rushing_yards_y,pacr_y,team_y,team_score_y,opponent_score_y
0,CIN,0.420724,0.041143,178.000000,38.500000,0.736746,0.205974,-1.016093,209.000000,20.722221,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0
1,DAL,-0.260247,-0.891837,116.125000,28.045454,0.935361,0.466962,-0.400317,155.571426,21.448277,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0
2,NYG,0.718937,0.389381,193.666672,34.409092,1.417280,-0.203076,-0.433977,139.428574,23.049999,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0
3,SF,-0.436521,-0.186934,183.666672,14.590909,1.926877,2.625062,0.009896,212.000000,31.280001,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0
4,WAS,1.528380,0.495720,251.800003,26.719999,0.862084,0.425777,0.113383,269.799988,23.947369,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0


In [57]:
#merge opponent stats on team

team_and_stats_test_data = team_and_stats.merge(team_and_stats_opp, on='opponent', how='right')
#team_and_stats_test_data = team_and_stats_test_data.drop(['team_score_y', 'opponent_score_y'], axis=1)
team_and_stats_test_data['opponent_score_x'] = team_and_stats_test_data['opponent_score_x']
team_and_stats_test_data['opponent_score_y'] = team_and_stats_test_data['opponent_score_y']
team_and_stats_test_data = team_and_stats_test_data.drop_duplicates()
#team_and_stats_test_data = team_and_stats_test_data.dropna(axis=1, how='any')

team_and_stats_test_data.head()

,team,defense_receiving_epa_x,defense_rushing_epa_x,defense_passing_yards_x,defensive_rushing_yards_x,d_pacr_x,receiving_epa_x,rushing_epa_x,passing_yards_x,rushing_yards_x,...,defensive_rushing_yards_y,d_pacr_y,receiving_epa_y,rushing_epa_y,passing_yards_y,rushing_yards_y,pacr_y,team_y,team_score_y,opponent_score_y
0,CIN,0.420724,0.041143,178.000000,38.500000,0.736746,0.205974,-1.016093,209.000000,20.722221,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0
1,DAL,-0.260247,-0.891837,116.125000,28.045454,0.935361,0.466962,-0.400317,155.571426,21.448277,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0
2,NYG,0.718937,0.389381,193.666672,34.409092,1.417280,-0.203076,-0.433977,139.428574,23.049999,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0
3,SF,-0.436521,-0.186934,183.666672,14.590909,1.926877,2.625062,0.009896,212.000000,31.280001,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0
4,WAS,1.528380,0.495720,251.800003,26.719999,0.862084,0.425777,0.113383,269.799988,23.947369,...,29.571428,1.12853,0.366136,0.11427,196.0,32.545456,0.84081,NYG,31.0,28.0


In [58]:
excel_file = 'test_data_ml.xlsx'
team_and_stats_test_data.to_excel(excel_file, index=False)  

In [66]:
column_names_test = [
    'defense_receiving_epa_x', 'defense_rushing_epa_x', 'defense_passing_yards_x',
    'd_pacr_x', 'receiving_epa_x', 'rushing_epa_x', 'passing_yards_x','pacr_x',
     'defense_receiving_epa_y', 'defense_rushing_epa_y',
    'defense_passing_yards_y','d_pacr_y', 'receiving_epa_y', 'rushing_epa_y',
    'passing_yards_y', 'pacr_y'
]

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# model
# Split data into features (X) and target variable (y)
X = team_and_stats_test_data[column_names_test]
y = team_and_stats_test_data[['team_score_x', 'opponent_score_x']]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

Mean Absolute Error (MAE): 6.571384896639905
Mean Squared Error (MSE): 67.24260866850004
Root Mean Squared Error (RMSE): 8.199342820473543
R-squared (R2): 0.40328320562762127


In [71]:
groupby_team = team_and_stats[team_and_stats['team'] == 'DEN']
groupby_team.mean()

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


defense_receiving_epa        1.983149
defense_rushing_epa          0.817700
defense_passing_yards      234.833328
defensive_rushing_yards     44.666668
d_pacr                       1.425492
receiving_epa                0.909429
rushing_epa                 -0.215516
passing_yards              242.000000
rushing_yards               27.421055
pacr                         1.082902
team_score                  24.200000
opponent_score              36.200000
dtype: float64

In [72]:
# Define the average values for each feature
average_values_model = {
    'defense_receiving_epa_x': -0.062288,
    'defense_rushing_epa_x': -0.479563,
    'defense_passing_yards_x': 182.666672,
    'd_pacr_x':  0.761711,
    'receiving_epa_x': 0.665469,
    'rushing_epa_x':-0.110006,
    'passing_yards_x': 219.666672,
    'pacr_x':  1.028912,
    'defense_receiving_epa_y': 1.983149,
    'defense_rushing_epa_y': 0.817700,
    'defense_passing_yards_y': 234.833328,
    'd_pacr_y':  1.425492,
    'receiving_epa_y': 0.909429,
    'rushing_epa_y': -0.215516,
    'passing_yards_y':  242.000000,
    'pacr_y': 1.082902
}


# Create a sample untrained dataset for X using the average values
data_untrained_test = {feature: [average_values_model[feature] for _ in range(4)] for feature in average_values_model}
X_untrained = pd.DataFrame(data_untrained_test)

In [73]:
# 2. Use the trained model to make predictions on X_untrained.
predictions_untrained_team = model.predict(X_untrained)
print(predictions_untrained_team)

[[33.49821231 18.06264584]
 [33.49821231 18.06264584]
 [33.49821231 18.06264584]
 [33.49821231 18.06264584]]


In [120]:
33-20

SyntaxError: invalid syntax (3868693530.py, line 1)